In [14]:
import requests

# URL of the FastAPI endpoint
# url = "http://localhost:8017/train"
url = "http://q80gcc4.103.167.198.31.sslip.io/train_test"

# Path to the Excel file
file_path = "/Volumes/DATA/Training_GPTs/Traindata GG_Full.xlsx"
# Read the file into a DataFrame
# Read the file into a DataFrame
with open(file_path, 'rb') as f:
    response = requests.post(url, files={'file': f}, data={'feature_cols': "DEPTH,CALI,DT,GR,LLD,LLS,NPHI,PHIE,RHOB,SW"})

if response.status_code == 200:
    print("Training successful:", response.json())
else:
    print("Error:", response.status_code, response.json())

Training successful: {'message': 'Model trained successfully', 'Mean Squared Error': 0.010053932949935467, 'R-squared Score': 0.8134363603202219}


In [13]:
import requests
import pandas as pd

# URL of the FastAPI prediction endpoint
# url = "http://localhost:8017/predict"

url ="http://q80gcc4.103.167.198.31.sslip.io/predict"

# Path to the Excel or CSV file
file_path = "Traindata_GG_for_Predict.xlsx"  # Change this to your file path

# Read the file into a DataFrame
if file_path.endswith('.xlsx'):
    data = pd.read_excel(file_path)
elif file_path.endswith('.csv'):
    data = pd.read_csv(file_path)
else:
    raise ValueError("Unsupported file format. Please use an Excel or CSV file.")

# Ensure the file contains the necessary columns
required_columns = ["DEPTH", "CALI", "DT", "GR", "LLD", "LLS", "NPHI", "PHIE", "RHOB", "SW"]
if not all(column in data.columns for column in required_columns):
    raise ValueError("Input file is missing required columns.")

# Function to make prediction for each row
def make_prediction(row):
    # Check for NaN values and replace them with a suitable default or handle them accordingly
    if row.isnull().any():
        print(f"Skipping row with NaN values: {row}")
        return None
    
    payload = {
        "DEPTH": row["DEPTH"],
        "CALI": row["CALI"],
        "DT": row["DT"],
        "GR": row["GR"],
        "LLD": row["LLD"],
        "LLS": row["LLS"],
        "NPHI": row["NPHI"],
        "PHIE": row["PHIE"],
        "RHOB": row["RHOB"],
        "SW": row["SW"]
    }
    response = requests.post(url, json=payload)
    if response.status_code == 200:
        return response.json().get("VWCL", None)
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

# Apply the function to each row in the DataFrame
data["VWCL_Prediction"] = data.apply(make_prediction, axis=1)

# Display the DataFrame with predictions
print(data)

# Optionally, save the DataFrame with predictions to a new file
output_file_path = "result_predictions.xlsx"  # Change this to your desired output file path
data.to_excel(output_file_path, index=False)
print(f"Predictions saved to {output_file_path}")

KeyboardInterrupt: 

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib

# Đọc dữ liệu từ file Excel
file_path = 'Traindata GG_Full.xlsx'  # Thay đổi đường dẫn tới file của bạn
data = pd.read_excel(file_path)

# Giả sử dữ liệu có các cột: 'feature1', 'feature2', 'feature3', ..., 'target'
# Thay đổi các cột dưới đây thành các cột có trong dữ liệu của bạn
selected_features = ["DEPTH", "CALI", "DT", "GR", "LLD", "LLS", "NPHI", "PHIE", "RHOB", "SW"]  # Thay đổi thành các feature bạn muốn sử dụng
target_column = 'VWCL'  # Thay đổi thành tên cột mục tiêu của bạn

# Xử lý các điểm null trong các feature
for feature in selected_features:
    if data[feature].isnull().sum() > 0:
        data[feature].fillna(data[feature].mean(), inplace=True)

# Xử lý các điểm null trong biến mục tiêu
if data[target_column].isnull().sum() > 0:
    data[target_column].fillna(data[target_column].mean(), inplace=True)

# Chia dữ liệu thành các biến đầu vào và biến mục tiêu
X = data[selected_features]
y = data[target_column]

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Khởi tạo và huấn luyện mô hình RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = model.predict(X_test)

# Đánh giá mô hình
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

# Lưu mô hình
model_path = 'model.pkl'
joblib.dump(model, model_path)
print(f'Model saved to {model_path}')


/var/folders/q_/yvjdxxl10s5fgncgwff3qlrm0000gn/T/ipykernel_36221/115012245.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[feature].fillna(data[feature].mean(), inplace=True)
/var/folders/q_/yvjdxxl10s5fgncgwff3qlrm0000gn/T/ipykernel_36221/115012245.py:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting val

Mean Squared Error: 0.0003260020274866867
R^2 Score: 0.9939506136460464
Model saved to model.pkl
